# `loop_calculator.py` — Algorithms & API

This document explains the **physics, numerical algorithms, and API** implemented in `loop_calculator.py`.

---

## 1) Physical model & assumptions

- Ideal gas, isothermal at temperature $T$.
- Steady, single-phase, subsonic flow (no choking).
- Each segment: length $L_i$, diameter $D_i$, roughness $\varepsilon_i$, minor loss $K_i$.
- Constant mass flow $\dot m$, local volumetric flow $Q(x)$ varies with $p(x)$.

**Ideal gas:**
$$
\rho = \frac{p}{R T}
$$

---

## 2) Tube model (isothermal, compressible)

Darcy–Weisbach:
$$
-\frac{dp}{dx} = f \frac{\rho v^2}{2D}, \quad v = \frac{\dot m}{\rho A}
$$

Integrating along a uniform segment:
$$
p_1^2 - p_2^2 = \frac{4 f L_\text{eff}}{D} \frac{R T}{A^2} \dot m^2
$$

with
- $A=\pi D^2/4$  
- $L_\text{eff} = L + \frac{K D}{f}$

**Reynolds number:**
$$
Re = \frac{\dot m D}{\mu A}
$$

**Friction factor:**
- Laminar ($Re<2300$): $f=64/Re$  
- Turbulent: Haaland explicit
$$
\frac{1}{\sqrt f} = -1.8 \log_{10}\!\Bigg[\Big(\frac{\varepsilon/D}{3.7}\Big)^{1.11} + \frac{6.9}{Re}\Bigg]
$$

---

## 3) Multi-segment propagation

Order segments: pump discharge → … → pump suction.

For trial $\dot m$ and pump outlet $p_\text{out}$:

1. Set $p^2 = p_\text{out}^2$.  
2. For each segment $i$, compute $\Delta p_i^2 = C_i \dot m^2$, then update  
   $p^2 \leftarrow p^2 - \Delta p_i^2$.  
3. After last segment: $p_\text{in} = \sqrt{p^2}$.

Thus
$$
p_\text{out}^2 - p_\text{in}^2 = \sum_i \Delta p_i^2(\dot m)
$$

---

## 4) Mean pressure in a closed loop

Volume-weighted mean is conserved:
$$
p_\text{mean} = \frac{1}{V} \int_V p(\mathbf r)\,dV
$$

For segment $i$:
$$
\bar p_i = \frac{2}{3}\,\frac{p_{i,\text{in}}^3 - p_{i,\text{out}}^3}{p_{i,\text{in}}^2 - p_{i,\text{out}}^2}
$$

Loop average:
$$
p_\text{avg} = \frac{\sum_i A_i L_i \bar p_i}{\sum_i A_i L_i}
$$

Closed-loop solver enforces $p_\text{avg} = p_\text{mean}$.

---

## 5) Pump model

**Overpressure curve:**
$$
Q(\Delta p) = Q_0 - s \Delta p
$$

**Vacuum curve:** $Q_\text{max}(p_\text{in})$ from datasheet.

Effective volumetric flow:
$$
Q_\text{eff}(\Delta p, p_\text{in}) = \min\!\big(Q(\Delta p),\,Q_\text{max}(p_\text{in})\big)
$$

Mass flow:
$$
\dot m_\text{pump} = \rho_\text{in} \frac{Q_\text{eff}}{1000\cdot 60}, \quad \rho_\text{in}=\frac{p_\text{in}}{RT}
$$

---

## 6) Solvers

### Open-case
Given $p_\text{in}, p_\text{out}$:

1. Tube-only $\dot m_\text{tube}$ from $p_\text{out}^2 - p_\text{in}^2 = \sum \Delta p_i^2(\dot m)$.  
2. Pump-only $\dot m_\text{pump}$ from pump curve.  
3. Result: $\dot m = \min(\dot m_\text{tube}, \dot m_\text{pump})$.

### Closed-loop (sealed, asymmetric)
Given $p_\text{mean}$:

Unknowns $p_\text{in}, \Delta p$, with $p_\text{out}=p_\text{in}+\Delta p$.  

Constraints:
1. Pump flow = loop flow.  
2. Volume-weighted mean $p_\text{avg}=p_\text{mean}$.

Solved by nested bisections:
- Outer on $\Delta p$, inner on $p_\text{in}$, innermost on $\dot m$.

---

## 7) API

- `Segment(L, D, eps, K=0.0)`  
- `PumpCurve(Q0_Lpm, s_Lpm_per_bar, vac_curve_pts)`  
  - `Q_overpressure(dp_bar)`  
  - `Qmax_vacuum(p_in_bar_abs)`  
  - `mdot(dp_bar, p_in_abs_Pa, T, R)`  
- `solve_open_case(p_in_bar_abs, p_out_bar_abs, segs, T, R, mu, pump)`  
- `solve_closed_loop_asym(p_mean_bar, segs, T, R, mu, pump)`

---

## 8) Usage example

```python
import loop_calculator as lc

T=293; R=63.3; mu=2.3e-5
pump=lc.PumpCurve(30,(30-20)/3,[(0.12,0.0),(0.20,5.0),(0.40,15.0),(1.00,30.0)])
segs=[lc.Segment(3.0,12.7e-3,0.0), lc.Segment(2.0,4.6e-3,0.0)]

print(lc.solve_closed_loop_asym(1.8, segs, T, R, mu, pump))